# Контрольная работа №1

Каждая задача решается в отдельном ноутбуке, код содержит комментарии, объясняющие вычисления.

Этот ноутбук подготовлен для запуска в Google Colab. Если вы откроете его в Colab — выполните клетки по порядку. Если какие-то файлы отсутствуют (diabetes_dataset.csv, image1.jpg), Colab предложит загрузить их вручную.

---

**Содержание ноутбука**:
1. Fourier-преобразование для `jena_climate_2009_2016.csv` (скачивание при необходимости), построение спектра плотности и восстановленного ряда.
2. PCA для `diabetes_dataset.csv`: предобработка, PCA, диаграмма первых 2 компонент с раскраской по `diagnosed_diabetes`.
3. Сжатие `image1.jpg` методом FFT и методом SVD: график сингулярных значений, спектра мощности, измерение среднего времени выполнения и объёма сжатых данных.


In [ ]:

# Часть 1 — Fourier-преобразование для jena_climate_2009_2016.csv
# Код автоматически скачивает dataset TensorFlow (если запущен в Colab) или пытается открыть локальный файл.
# Мы будем работать с температурой 'T (degC)'.

import os
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import welch

# Если запускаете в Colab — можно скачать архив с датасетом TensorFlow. Если нет — загрузите файл вручную.
jena_fname = 'jena_climate_2009_2016.csv'
if not os.path.exists(jena_fname):
    try:
        print('Файл не найден локально — пытаюсь скачать архив TensorFlow...')
        !wget -q https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip -O jena.zip
        !unzip -o jena.zip -d .
        if not os.path.exists(jena_fname):
            raise Exception('Не удалось скачать файл автоматически. Загрузите jena_climate_2009_2016.csv вручную.')
        else:
            print('Файл скачан и распакован.')
    except Exception as e:
        print('Автоматическое скачивание не удалось:', e)
        from google.colab import files
        print('Пожалуйста, загрузите jena_climate_2009_2016.csv вручную.')
        uploaded = files.upload()  # запустится в Colab
        for k in uploaded.keys():
            print('Загружен', k)

# Загружаем csv
df = pd.read_csv(jena_fname)
# Показываем колонки — чтобы найти нужную колонку температуры
print('Колонки датасета:', df.columns.tolist())

# Обычно в этом датасете колонка с температурой называется 'T (degC)'
temp_col = None
candidates = ['T (degC)', 'T', 'Air temperature (degC)']
for c in candidates:
    if c in df.columns:
        temp_col = c
        break
if temp_col is None:
    # если структура другая — возьмём первую числовую колонку
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    temp_col = numeric_cols[0]
    print('Не найдена стандартная колонка температуры, используем', temp_col)
else:
    print('Используем температурную колонку:', temp_col)

signal = df[temp_col].astype(float).values
times = pd.to_datetime(df['Date Time']) if 'Date Time' in df.columns else None

# Для анализа возьмём более короткий фрагмент (примерно 10 000 точек) чтобы быстрее выполнять демонстрацию
n = min(len(signal), 20000)
signal = signal[:n]
if times is not None:
    times = times[:n]

# Убираем среднее — для спектрального анализа
signal_mean = signal.mean()
signal_centered = signal - signal_mean

# Быстрое преобразование Фурье
N = len(signal_centered)
yf = fft(signal_centered)
xf = fftfreq(N, d=1.0)  # в данном датасете шаг времени — 10 минут, но используем d=1 условно
power = np.abs(yf)**2 / N

# Оценка спектра мощности с помощью Welch для сравнения
f_welch, Pxx = welch(signal_centered, nperseg=1024)

# Строим график спектра плотности мощности и временного ряда до/после фильтрации
import matplotlib.pyplot as plt
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(f_welch[:len(f_welch)//10], Pxx[:len(Pxx)//10])
plt.title('PSD (Welch) — низкочастотная часть')
plt.xlabel('Частота (условн.)')
plt.ylabel('Мощность')

plt.subplot(1,2,2)
plt.semilogy(np.fft.fftshift(fftfreq(N)), np.fft.fftshift(power))
plt.title('Полный спектр мощности (лог шкала)')
plt.xlabel('Частота (условн.)')
plt.ylabel('Мощность (лог)')
plt.tight_layout()
plt.show()

# Фильтрация: оставим только низкие частоты — например, уберём компоненты с индексами, чей модуль больше cutoff
# Подход: выбрать порог по доле полной мощности (например, сохранить 90% мощности в низкочастотной области),
# или выбрать фиксированный cutoff (например, топ k низкочастотных компонент).
# Здесь реализуем простой метод: сохраняем компоненты с |freq_index| < k_low
k_low = int(0.01 * N)  # оставляем 1% нижних частот по индексам
mask = np.zeros(N, dtype=bool)
mask[:k_low] = True
mask[-k_low:] = True  # симметрично
yf_filtered = yf * mask
reconstructed = np.real(ifft(yf_filtered)) + signal_mean

# Построим оригинал и реконструкцию
plt.figure(figsize=(12,5))
plt.plot(signal, label='Оригинал (фрагмент)')
plt.plot(reconstructed, label=f'Реконструкция (низкие частоты, k_low={k_low})', linewidth=2)
plt.legend()
plt.title('Временной ряд: до и после фильтрации FFT')
plt.xlabel('Индекс времени')
plt.tight_layout()
plt.show()

# Покажем долю мощности, сохранённой после фильтрации
power_total = power.sum()
power_filtered = (np.abs(yf_filtered)**2 / N).sum()
print(f'Доля мощности, сохранённой низкими частотами: {power_filtered/power_total:.4f}')


In [ ]:

# Часть 2 — PCA для diabetes_dataset.csv
# Ожидается, что файл diabetes_dataset.csv загружен в рабочую папку. Если нет — Colab предложит загрузить.
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

fname = 'diabetes_dataset.csv'
if not os.path.exists(fname):
    try:
        from google.colab import files
        print('Файл diabetes_dataset.csv не найден. Загрузите его сейчас.')
        uploaded = files.upload()
        for k in uploaded.keys():
            print('Загружен', k)
    except Exception as e:
        print('Запуск вне Colab — убедитесь, что diabetes_dataset.csv находится в текущей папке.')

df = pd.read_csv(fname)
print('Первые строки датасета:')
display(df.head())

# Найдём целевой столбец 'diagnosed_diabetes' (возможны варианты регистра)
target_cols = [c for c in df.columns if c.lower()=='diagnosed_diabetes' or c.lower().endswith('diagnosed_diabetes')]
if not target_cols:
    # Попробуем варианты: 'diabetes', 'Outcome', 'target'
    for alt in ['diabetes','Outcome','target','Diagnosis','diagnosed']:
        if alt in df.columns:
            target_cols = [alt]; break
if not target_cols:
    raise ValueError('Не найден столбец с метками диабета. Пожалуйста, проверьте название столбца (ожидается diagnosed_diabetes).')

target_col = target_cols[0]
print('Используем столбец меток:', target_col)

# Предобработка: преобразуем категориальные признаки в числа (LabelEncoder) или удалим сильно текстовые столбцы
X = df.drop(columns=[target_col]).copy()
y = df[target_col].copy()

# Простейшая обработка: для нечисловых столбцов применим LabelEncoder (потенциально лучше — one-hot, но для PCA допускается кодировка целыми числами)
for c in X.columns:
    if X[c].dtype == 'object' or X[c].dtype.name == 'category':
        le = LabelEncoder()
        X[c] = le.fit_transform(X[c].astype(str))

# Удалим колонки с Nan или заменим средним
X = X.fillna(X.mean())

# Масштабирование перед PCA
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# Применяем PCA и выводим первые 2 компоненты
pca = PCA(n_components=2)
pcs = pca.fit_transform(Xs)

print('Доля объяснённой дисперсии по компонентам:', pca.explained_variance_ratio_)

# Визуализация: первые 2 компоненты
plt.figure(figsize=(8,6))
# Преобразуем метки в 0/1 если необходимо
try:
    y_num = pd.to_numeric(y, errors='coerce')
    mask_nan = np.isnan(y_num)
    if mask_nan.any():
        # если не получилось — применим LabelEncoder
        from sklearn.preprocessing import LabelEncoder
        y_num = LabelEncoder().fit_transform(y.astype(str))
except Exception:
    from sklearn.preprocessing import LabelEncoder
    y_num = LabelEncoder().fit_transform(y.astype(str))

plt.scatter(pcs[:,0], pcs[:,1], c=y_num, cmap='viridis', s=20, alpha=0.7)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA: первые 2 главные компоненты (цвет = diagnosed_diabetes)')
plt.colorbar(label='diagnosed_diabetes (код)')
plt.grid(True)
plt.show()


In [ ]:

# Часть 3 — Сжатие изображения image1.jpg: FFT и SVD
# Ожидается файл image1.jpg в рабочей папке; если нет — Colab предложит загрузить.
import os, time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from scipy.linalg import svd
from timeit import default_timer as timer

img_fname = 'image1.jpg'
if not os.path.exists(img_fname):
    try:
        from google.colab import files
        print('Файл image1.jpg не найден. Загрузите его сейчас.')
        uploaded = files.upload()
        for k in uploaded.keys():
            print('Загружен', k)
    except Exception as e:
        print('Запуск вне Colab — убедитесь, что image1.jpg находится в текущей папке.')

img = Image.open(img_fname).convert('RGB')
arr = np.array(img).astype(float)/255.0  # нормируем для обработки
h,w,c = arr.shape
print(f'Изображение: {img_fname}, размер {w}x{h}, каналы: {c}')

# Функция FFT-сжатия: обнуляем малую часть спектра (по доле мощности)
def fft_compress(channel, keep_fraction=0.1):
    # channel: 2D array
    F = fft2(channel)
    magnitude = np.abs(F)
    # оставим только верхние по модулю коэффициенты
    flat = magnitude.flatten()
    threshold = np.sort(flat)[-int(len(flat)*keep_fraction)] if keep_fraction>0 else np.inf
    mask = magnitude >= threshold
    F_comp = F * mask
    rec = np.real(ifft2(F_comp))
    return rec, mask

# Функция SVD-сжатия на каждом канале: оставляем k сингулярных значений
def svd_compress(channel, k):
    U, s, Vt = svd(channel, full_matrices=False)
    S = np.zeros((k,k))
    S[:k,:k] = np.diag(s[:k])
    rec = (U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:])
    return rec, s

# Эксперимент: измерим среднее время и объём сжатых данных для нескольких параметров
fft_results = {}
svd_results = {}

ks = [5, 20, 50]  # для SVD
keep_fracs = [0.01, 0.05, 0.1]  # для FFT

# повторов для усреднения
repeats = 3

for keep in keep_fracs:
    t0 = timer()
    masks = []
    rec_channels = []
    for ch in range(3):
        total_t = 0.0
        for r in range(repeats):
            start = timer()
            rec, mask = fft_compress(arr[:,:,ch], keep_fraction=keep)
            total_t += (timer() - start)
        rec_channels.append(rec)
        masks.append(mask)
    t_avg = total_t / repeats
    rec_img = np.clip(np.stack(rec_channels,axis=2),0,1)
    # вычислим объём сжатых данных: количество ненулевых коэффициентов в F для всех каналов
    nonzeros = sum([m.sum() for m in masks])
    # Оценим "объём" в байтах: число комплексных коэффициентов * 16 байт (double для реальной и мнимой части)
    est_bytes = nonzeros * 16
    fft_results[keep] = {'time_avg': t_avg, 'nonzeros': int(nonzeros), 'est_bytes': int(est_bytes), 'reconstruction': rec_img}

for k in ks:
    total_t = 0.0
    rec_channels = []
    nonzeros = 0
    singular_values_all = []
    for ch in range(3):
        # усредняем по повтору выполнения SVD (SVD сам по себе детерминированный)
        start = timer()
        rec, s = svd_compress(arr[:,:,ch], k)
        total_t += (timer() - start)
        rec_channels.append(rec)
        singular_values_all.append(s)
        nonzeros += (arr.shape[0]*k + k + k*arr.shape[1])  # числа параметров U (n*k) + diag(k) + Vt (k*m)
    t_avg = total_t  # для SVD не делаем многократных повтора (дорого), оставляем время 1 прохода
    rec_img = np.clip(np.stack(rec_channels,axis=2),0,1)
    est_bytes = nonzeros * 8  # приблизительно, double 8 байт на число
    svd_results[k] = {'time': t_avg, 'params': int(nonzeros), 'est_bytes': int(est_bytes), 'reconstruction': rec_img, 'singular_values': singular_values_all}

# Показ результатов: оригинал, FFT-реконструкция (пример), SVD-реконструкция (пример)
plt.figure(figsize=(12,6))
plt.subplot(2,3,1); plt.imshow(arr); plt.title('Оригинал'); plt.axis('off')
# выберем keep=0.05 для демонстрации
k_example = ks[1]
keep_example = keep_fracs[1]
plt.subplot(2,3,2); plt.imshow(fft_results[keep_example]['reconstruction']); plt.title(f'FFT rec keep={keep_example}'); plt.axis('off')
plt.subplot(2,3,3); plt.imshow(svd_results[k_example]['reconstruction']); plt.title(f'SVD rec k={k_example}'); plt.axis('off')

# график сингулярных значений (для первого канала)
svals = svd_results[k_example]['singular_values'][0]
plt.subplot(2,3,4); plt.plot(svals, marker='o'); plt.title('Сингулярные значения (канал 0)'); plt.xlabel('i'); plt.ylabel('s_i')
plt.subplot(2,3,5); plt.semilogy(svals, marker='o'); plt.title('Сингулярные значения (лог шкала)'); plt.xlabel('i')
plt.subplot(2,3,6)
# спектр мощности канала 0
F = fft2(arr[:,:,0])
power = np.abs(F)**2
plt.imshow(fftshift(np.log1p(power)), aspect='auto')
plt.title('Спектр мощности (лог) канала 0'); plt.axis('off')

plt.tight_layout()
plt.show()

# Выведем таблицу с результатами оценки времени и объёма
import pandas as pd
rows = []
for keep,v in fft_results.items():
    rows.append({'method':'FFT','param':f'keep={keep}','time_avg_s':v['time_avg'],'nonzeros':v['nonzeros'],'est_bytes':v['est_bytes']})
for k,v in svd_results.items():
    rows.append({'method':'SVD','param':f'k={k}','time_avg_s':v['time'],'nonzeros':v['params'],'est_bytes':v['est_bytes']})
df_res = pd.DataFrame(rows).sort_values(['method','param'])
display(df_res)

# Сохраним примеры реконструкций
from PIL import Image
def save_img(arr_norm, fname):
    im = Image.fromarray((np.clip(arr_norm,0,1)*255).astype('uint8'))
    im.save(fname)

save_img(fft_results[keep_example]['reconstruction'], 'fft_reconstruction_example.jpg')
save_img(svd_results[k_example]['reconstruction'], 'svd_reconstruction_example.jpg')
print('Сохранены примеры: fft_reconstruction_example.jpg, svd_reconstruction_example.jpg')


---

**Как использовать ноутбук**:

1. Откройте `Control_Work1_Colab.ipynb` в Google Colab.
2. Запустите клетки сверху вниз. Если Colab запросит загрузить `diabetes_dataset.csv` или `image1.jpg`, загрузите их.
3. Просмотрите графики и сохранённые примеры реконструкций (`fft_reconstruction_example.jpg`, `svd_reconstruction_example.jpg`).

Если хотите, могу сейчас сохранить этот ноутбук и дать ссылку для скачивания. Обычно файл появится по пути `/mnt/data/Control_Work1_Colab.ipynb`.
